#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Ghaati")
loader

In [4]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Ghaati', 'title': 'Ghaati - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nGhaati - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nPlot\n\n\n\n\n

In [5]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [6]:
documents

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Ghaati', 'title': 'Ghaati - Wikipedia', 'language': 'en'}, page_content='Ghaati - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nPlot\n\n\n\n\n\n\n\n\n

In [7]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

c:\Users\DeLL\Documents\GitHub\OpenAI-and-Ollama\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [9]:
vectorstoredb

In [10]:
## Query From a vector db
query="On 19 March 2024, the project was announced at an Amazon Prime Video"
result=vectorstoredb.similarity_search(query)
result[0].page_content

"VTV Ganesh as M.P. Bojjayya\nJwala Koti as Koti\nSatya as Bhupesh\nRaju Sundaram as Pebbuli\nRavi Mariya as Siddhappa Naidu\nProduction[edit]\nDevelopment[edit]\nOn 19 March 2024, the project was announced at an Amazon Prime Video showcase event with the release of the first look.[6][7] On 7 November 2024, the makers released a Poster and 47-second glimpse of the film on the occasion of Anushka Shetty's birthday.[8][9][10] The makers announced the release date of the film with an intriguing video on 15 December 2024.[11][12]"

In [11]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [12]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023B0B7EC2D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023B0C418D90>, root_client=<openai.OpenAI object at 0x0000023B0C40EA50>, root_async_client=<openai.AsyncOpenAI object at 0x0000023B0C40CD90>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, confi

In [13]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"On 19 March 2024, the project was announced at an Amazon Prime Video",
    "context":[Document(page_content="On 19 March 2024, the project was announced at an Amazon Prime Video showcase event with the release of the first look.[6][7] On 7 November 2024, the makers released a Poster and 47-second glimpse of the film on the occasion of Anushka Shetty's birthday.[8][9][10] The makers announced the release date of the film with an intriguing video on 15 December 2024.[11][12]")]
})

"Based on the provided context, what can be inferred about the events surrounding the film's release announcements?\n\n1. The project related to the film was first announced on 19 March 2024 during an Amazon Prime Video showcase event, along with the release of the film's first look.\n\n2. On 7 November 2024, in celebration of Anushka Shetty's birthday, the filmmakers released a poster and a 47-second glimpse of the film.\n\n3. The official release date of the film was announced on 15 December 2024, accompanied by an intriguing video."

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [14]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [15]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [16]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023B0C9F7D50>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [17]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

'What information is provided about the plot of "Ghaati"?\n\nThe plot section mentions "The story of a victim who turns into a criminal and later becomes a legend." It also notes that this section needs expansion as of September 2025.'

In [18]:

response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(id='a854188d-798a-4f55-b438-96285ad85fd5', metadata={'source': 'https://en.wikipedia.org/wiki/Ghaati', 'title': 'Ghaati - Wikipedia', 'language': 'en'}, page_content='Privacy policy\nAbout Wikipedia\nDisclaimers\nContact Wikipedia\nCode of Conduct\nDevelopers\nStatistics\nCookie statement\nMobile view\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\nToggle the table of contents\n\n\n\n\n\n\n\nGhaati\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1 language\n\n\nAdd topic'),
  Document(id='9c033286-0ffe-4a18-8235-b140e06b8b81', metadata={'source': 'https://en.wikipedia.org/wiki/Ghaati', 'title': 'Ghaati - Wikipedia', 'language': 'en'}, page_content='Plot[edit]\nThis section needs expansion with: September 2025. You can help by adding to it.  (September 2025)\nThe story of a victim who turns into a criminal and later becomes a legend.'),
  D

In [19]:
response['context']

[Document(id='a854188d-798a-4f55-b438-96285ad85fd5', metadata={'source': 'https://en.wikipedia.org/wiki/Ghaati', 'title': 'Ghaati - Wikipedia', 'language': 'en'}, page_content='Privacy policy\nAbout Wikipedia\nDisclaimers\nContact Wikipedia\nCode of Conduct\nDevelopers\nStatistics\nCookie statement\nMobile view\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\nToggle the table of contents\n\n\n\n\n\n\n\nGhaati\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1 language\n\n\nAdd topic'),
 Document(id='9c033286-0ffe-4a18-8235-b140e06b8b81', metadata={'source': 'https://en.wikipedia.org/wiki/Ghaati', 'title': 'Ghaati - Wikipedia', 'language': 'en'}, page_content='Plot[edit]\nThis section needs expansion with: September 2025. You can help by adding to it.  (September 2025)\nThe story of a victim who turns into a criminal and later becomes a legend.'),
 Document(id='c041d7df-7e26-40a3-931f-6ff6903d7a38', metadata={'source': 'https://en.wi